<a href="https://colab.research.google.com/github/yorkjong/stock-reports/blob/main/notebooks/weinstein_reports.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Stock Screening Inspired by Stan Weinstein

This notebook features stock screening methods inspired by Stan Weinstein's book, *Secrets for Profiting in Bull and Bear Markets*. The strategies and criteria used in this analysis are based on the principles outlined in Weinstein's work, aiming to identify potential investment opportunities in both bull and bear markets.


### Install and Setup (this section will be executed automatically)

#### Install Required Packages

In [96]:
%pip install "git+https://github.com/yorkjong/vistock.git@feature/mansfield"
%pip install requests-cache

!wget -O NotoSansTC-Regular.ttf https://share.cole.tw/d/Tools%20-%20MAC/Fonts/Noto_Sans_TC/static/NotoSansTC-Regular.ttf?sign=bATsZP5QZdI_2EM15sAbcAE48Cacle91CpwUNOCMuM8=:0

  Cloning https://github.com/yorkjong/vistock.git (to revision feature/mansfield) to /tmp/pip-req-build-dlwum1oj
  Running command git clone --filter=blob:none --quiet https://github.com/yorkjong/vistock.git /tmp/pip-req-build-dlwum1oj
  Running command git checkout -b feature/mansfield --track origin/feature/mansfield
  Switched to a new branch 'feature/mansfield'
  Branch 'feature/mansfield' set up to track remote branch 'feature/mansfield' from 'origin'.
  Resolved https://github.com/yorkjong/vistock.git to commit c31f4b2ec050e4709e3e39fd5c2a0118d9c6474e
  Preparing metadata (setup.py) ... done
--2024-09-16 19:05:05--  https://share.cole.tw/d/Tools%20-%20MAC/Fonts/Noto_Sans_TC/static/NotoSansTC-Regular.ttf?sign=bATsZP5QZdI_2EM15sAbcAE48Cacle91CpwUNOCMuM8=:0
Resolving share.cole.tw (share.cole.tw)... 180.218.0.193
Connecting to share.cole.tw (share.cole.tw)|180.218.0.193|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7110560 (6.8M) [application/octet-stream

In [97]:
# @title Install Cache
import requests_cache
requests_cache.install_cache('ibd_cache', expire_after=3600)

#### Setup and Configuration

In [98]:
# @title Enable Requests Cache
import requests_cache
requests_cache.install_cache('ibd_cache', expire_after=3600)

In [99]:
# @title Set Chinese Font for matplotlib
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt

fm.fontManager.addfont('NotoSansTC-Regular.ttf')
font_name = 'Noto Sans TC'
if font_name not in plt.rcParams['font.sans-serif']:
    plt.rcParams['font.sans-serif'].insert(0, font_name)

In [100]:
# @title ReadOnlyGitHub
import requests
import pandas as pd
from io import StringIO

class ReadOnlyGitHub:
    def __init__(self, repo_owner, repo_name, dir='', branch='main'):
        dir = dir.strip('/')
        base = 'https://raw.githubusercontent.com'
        if dir:
            self.raw_url = f'{base}/{repo_owner}/{repo_name}/{branch}/{dir}'
        else:
            self.raw_url = f'{base}/{repo_owner}/{repo_name}/{branch}'

        base = 'https://api.github.com/repos'
        if dir:
            self.api_url = f'{base}/{repo_owner}/{repo_name}/contents/{dir}'
        else:
            self.api_url = f'{base}/{repo_owner}/{repo_name}/contents'

    def file_exists(self, file_path):
        url = f'{self.raw_url}/{file_path}'
        response = requests.head(url)
        return response.status_code == 200

    def list_filenames(self, dir_path=''):
        url = f'{self.api_url}/{dir_path}'
        response = requests.get(url)
        if response.status_code == 200:
            files = response.json()
            return [item['name'] for item in files]
        else:
            print(f"Request failed: {response.status_code} - {response.text}")
            return []

    def download_csv(self, file_path):
        url = f'{self.raw_url}/{file_path}'
        if self.file_exists(file_path):
            return pd.read_csv(url)
        else:
            return pd.DataFrame()

# Create a GitHub instance
github = ReadOnlyGitHub(
    repo_owner='YorkJong',
    repo_name='stock-reports',
    dir='weinstein'
)

In [101]:
# @title DataFrame Utilities

def print_column(df, column):
    if column in df.columns:
        print(', '.join(df[column]))

In [102]:
# @title Source of Tickers

def tickers_from_df(df):
    if 'Name' in df.columns:
        return list(df['Name'])
    elif 'Ticker' in df.columns:
        return list(df['Ticker'])
    return []

def sorting_tickers():
    return list(tickers_from_df(df_top_f1))

def increasing_rs_tickers():
    return list(tickers_from_df(df_top_f2))

def rs_breakout_tickers():
    return list(tickers_from_df(df_top_f3))

def major_indices():
    return ['^DJI', '^IXIC', '^NDX', '^RUT', '^SOX',
            '^TWII', '^N225', '^HSI',
            '^STOXX50E', '^FTSE', '^GDAXI', '^FCHI', '^GSPTSE']

def sector_indices():
    return ['SOXX', 'DVY',
            'IWB','IWM', 'IWV',  'IJR',
            'ITB', 'IHI', 'IYC', 'ITA', 'IAK',
            'IYZ', 'IYT', 'IYR', 'IYF', 'IYJ',
            'IYG', 'IYH', 'IYK', 'IDU', 'IYE', 'IHE',
            'IAT', 'IAI', 'IEO', 'IYM', 'IHF']

In [103]:
# @title Checkboxes

import ipywidgets as widgets

def cbs_create(symbols, n_pre_checked=10):
    '''Create a list of checkboxes'''
    return [
        widgets.Checkbox(
            value=(i < n_pre_checked),  # Set first n items as checked
            description=symbol,
            layout=widgets.Layout(width='auto'),
            style={'description_width': 'auto'}
        )
        for i, symbol in enumerate(symbols)
    ]

def cbs_with_grid(checkboxes, n_cols=5):
    '''Create a grid layouting the given checkboxes'''
    return widgets.GridBox(checkboxes, layout=widgets.Layout(
        width='auto',
        grid_template_columns=f'repeat({n_cols}, 1fr)',
        grid_gap='10px'  # Add some space between the checkboxes
    ))

def cbs_get_selected(checkboxes):
    '''Get the selected symbols from the given checkboxes'''
    return [checkbox.description for checkbox in checkboxes if checkbox.value]

def cbs_unselect_all(checkboxes):
    '''Unselect all checkboxes in the given list'''
    for checkbox in checkboxes:
        checkbox.value = False

def cbs_select_top(checkboxes, n=10):
    '''Select the top n checkboxes in the given list'''
    for i, checkbox in enumerate(checkboxes):
        checkbox.value = (i < n)

In [104]:
# @title Dropdown Menus

def create_period_dropdown(value='2y'):
    return widgets.Dropdown(
        options=['1y', '2y', '5y'],
        value=value,
        description='Period:',
    )

def create_interval_dropdown(value='1wk'):
    return widgets.Dropdown(
        options=['1d', '1wk'],
        value=value,
        description='Interval:',
    )

def create_style_dropdown(desc=None, value=None):
    return widgets.Dropdown(
        options=['default', 'classic', 'yahoo', 'charles', 'tradingview', 'binance', 'binancedark', 'mike', 'nightclouds', 'checkers', 'ibd', 'sas', 'starsandstripes', 'kenan', 'blueskies', 'brasil'],
        value='yahoo' if value is None else value,
        description='Style:' if desc is None else desc,
        style={'description_width': 'initial'},
    )

def create_template_dropdown(desc=None, value=None):
    return widgets.Dropdown(
        options=['plotly', 'plotly_white', 'plotly_dark', 'ggplot2', 'seaborn', 'simple_white', 'presentation', 'xgridoff', 'ygridoff'],
        value='plotly_dark' if value is None else value,
        description='Template:' if desc is None else desc,
        style={'description_width': 'initial'},
    )

In [105]:
# @title Multiple Searchable Dropdown Menus
import ipywidgets as widgets

def create_search_box():
    '''Create a Text widget for search input'''
    return widgets.Text(
        description='Search:',
        placeholder='Type to search',
        layout=widgets.Layout(width='auto')
    )

def create_dropdown(options, description='Stock:'):
    '''Create a Dropdown widget for displaying filtered options'''
    return widgets.Dropdown(
        description=description,
        options=[None] + options,  # None as the default option
        layout=widgets.Layout(width='auto'),
        value=None  # Set default value to None
    )

def update_dropdown(change, dropdown, options):
    '''Update the options in the dropdown based on search input'''
    search_text = change['new'].lower()
    filtered = [option for option in options if search_text in option.lower()]
    if filtered:
        dropdown.options = [None] + filtered
        dropdown.value = filtered[0]  # Auto-select the first matching option
    else:
        dropdown.options = [None]  # Retain only the None option if no match

def remove_duplicates_preserve_order(lst):
    '''Remove duplicates from a list while preserving order'''
    seen = set()
    result = []
    for item in lst:
        if item and item not in seen:
            seen.add(item)
            result.append(item)
    return result

def get_dropdowns_selected_options(dropdowns):
    '''Get selected options from the dropdowns'''
    selected = [dropdown.value for dropdown in dropdowns if dropdown.value]
    return remove_duplicates_preserve_order(selected)

def create_search_dropdowns(options, max_selections):
    '''Create a layout with search boxes and dropdowns'''
    # Create UI components
    search_boxes = [create_search_box() for _ in range(max_selections)]
    dropdowns = [create_dropdown(options) for _ in range(max_selections)]

    # Bind search box and dropdown menu events
    for search_box, dropdown in zip(search_boxes, dropdowns):
        search_box.observe(lambda change, dropdown=dropdown:
                            update_dropdown(change, dropdown, options),
                            names='value')
    # Create the layout
    controls = [widgets.HBox([search_box, dropdown])
                for search_box, dropdown  in zip(search_boxes, dropdowns)]
    layout = widgets.VBox(controls)
    return dropdowns, layout

In [106]:
# @title Outputs
outputs = widgets.VBox()

In [107]:
# @title enable_plotly_in_cell
# ref. https://stackoverflow.com/questions/76593068/plotly-figure-not-rendering-in-ipywidgets-interact-function-google-colab
def enable_plotly_in_cell():
  import IPython
  from plotly.offline import init_notebook_mode
  display(IPython.core.display.HTML('''<script src="/static/components/requirejs/require.js"></script>'''))
  init_notebook_mode(connected=False)

### Glossary of Terms

#### Source (The Source of Stocks to Analyze)
- **Description**: Stocks can be sourced from various exchanges or indices.
- **Common Abbreviations**:
  - **Taiwan Markets**:
    - `TWSE`: Taiwan Stock Exchange (台灣上市股票交易所)
    - `TPEX`: Taipei Exchange (上櫃交易所)
    - `ESB`: Emerging Stock Board (興櫃交易所)
    - Combine with `+` (e.g., `TWSE+TPEX`, `TWSE+TPEX+ESB`)
  - **America Markets**:
    - `SPX`: S&P 500 (標普五百指數)
    - `DJIA`: Dow Jones Industrial Average (道瓊指數)
    - `NDX`: NASDAQ-100 (納斯達克一百指數)
    - `RUI`: Russell 1000
    - `RUT`: Russell 2000
    - `SOX`: PHLX Semiconductor Index (費半指數)
    - `W5000`: Wilshire 5000 Total Market Index
    - `USLS`: U.S. Listed Stocks (美國上市股票)
    - `U.S.Listed`: U.S. Listed Stocks (美國上市股票)
    - Combine with `+` (e.g., `SPX+DJIA+NDX+SOX`)

#### Period (Historical Data Time Range)
- **Description**: The duration for fetching historical data.
- **Example**: `2y` for 2 years

#### Interval (Historical Data Frequency)
- **Description**: The frequency of historical data points.
- **Common Intervals**:
  - `'1d'`: Daily data
  - `'1wk'`: Weekly data
  - `'1mo'`: Monthly data

#### MA (Moving Average)
- **SMA**: Simple Moving Average
- **EMA**: Exponential Moving Average

#### RS (Relative Strength)
- **Description**: Measures a stock's performance relative to a benchmark index.
- **Unit**: Percentage (%)
- **Interpretation**:
  - A value of 0 represents the performance of the benchmark index or market.
  - A positive value indicates outperformance relative to the benchmark.
  - A negative value indicates underperformance relative to the benchmark.

#### EPS RS (EPS Relative Strength)
- **Description**: Measures a stock's earnings per share (EPS) performance relative to a benchmark index.
- **Unit**: Percentage (%)
- **Interpretation**:
  - A value of 0 represents the EPS performance of the benchmark index.
  - A positive value indicates that the stock's EPS growth has outperformed the benchmark.
  - A negative value indicates underperformance relative to the benchmark.
- **Index EPS Calculation**: Weighted according to component stocks' share counts.

#### Rev RS (Operating Revenue Relative Strength)
- **Description**: Measures a stock's operating revenue performance relative to a benchmark index.
- **Unit**: Percentage (%)
- **Interpretation**:
  - A value of 0 represents the revenue performance of the benchmark index.
  - A positive value indicates that the stock's revenue growth has outperformed the benchmark.
  - A negative value indicates underperformance relative to the benchmark.
- **Index Revenue Calculation**: Weighted according to component stocks' market capitalization.

#### TTM EPS (Trailing Twelve Months EPS)
- **Description**: The earnings per share over the trailing twelve months, which gives a more recent view of the company's profitability.
- **Unit**: Currency per share
- **Interpretation**:
  - A higher TTM EPS indicates better profitability over the past year.
  - Used to compare with previous periods or with other companies.

#### TTM RPS (Trailing Twelve Months Revenue Per Share)
- **Description**: The revenue per share over the trailing twelve months, showing how much revenue a company generates per share.
- **Unit**: Currency per share
- **Interpretation**:
  - A higher TTM RPS indicates better revenue generation capability per share over the past year.
  - Useful for comparing revenue performance with previous periods or with other companies.

#### TTM PE (Trailing Twelve Months PE)
- **Description**: The price-to-earnings ratio calculated using the earnings from the trailing twelve months. It provides a measure of how much investors are willing to pay per dollar of earnings.
- **Unit**: Ratio
- **Interpretation**:
  - A higher TTM PE indicates that investors expect higher future growth and are willing to pay more for the stock.
  - A lower TTM PE might suggest undervaluation or lower expected growth.


##### References:
- [Mansfield Relative Strength (Original Version) by stageanalysis — Indicator by Stage_Analysis — TradingView](https://www.tradingview.com/script/NzUBDDtb-Mansfield-Relative-Strength-Original-Version-by-stageanalysis/)
- [Mansfield Relative Strength | TrendSpider Store](https://trendspider.com/trading-tools-store/indicators/mansfield-relative-strength/)
- [Mansfield Relative Strength | ChartMill.com](https://www.chartmill.com/documentation/technical-analysis/indicators/35-Mansfield-Relative-Strength)
- [How to create the Mansfield Relative Performance Indicator - Stage Analysis](https://www.stageanalysis.net/blog/4266/how-to-create-the-mansfield-relative-performance-indicator)
  - [Stan Weinstein's Stage Analysis | Page 49 | Trade2Win Forums • UK Financial Trading Community](https://www.trade2win.com/threads/stan-weinsteins-stage-analysis.134944/page-49#post-2137398)


## Execute Actions Step by Step


In [76]:
# @title Step 1. Pick a File

import ipywidgets as widgets

with requests_cache.disabled():
    filenames = github.list_filenames()

# Extract and sort all unique values in reverse order
all_dates = sorted(set(fn.split('_')[4].replace('.csv', '') for fn in filenames), reverse=True)
all_sources = sorted(set(fn.split('_')[0] for fn in filenames))
all_types = sorted(set(fn.split('_')[1] for fn in filenames), reverse=True)
all_periods = sorted(set(fn.split('_')[2] for fn in filenames), reverse=True)

# Calculate the maximum length of filenames to set dropdown width
max_filename_length = max(len(fn) for fn in filenames)
dropdown_width = f'{max_filename_length * 10}px'  # 10px width per character

# Create dropdowns with dynamic width
date_dropdown = widgets.Dropdown(
    options=all_dates,
    description='Date:',
    layout=widgets.Layout(width=dropdown_width)
)

source_dropdown = widgets.Dropdown(
    options=all_sources,
    description='Source:',
    layout=widgets.Layout(width=dropdown_width)
)

type_dropdown = widgets.Dropdown(
    options=all_types,
    description='Type:',
    layout=widgets.Layout(width=dropdown_width)
)

period_dropdown = widgets.Dropdown(
    options=all_periods,
    description='Period:',
    layout=widgets.Layout(width=dropdown_width)
)

# Create file selection dropdown
file_dropdown = widgets.Dropdown(
    options=[],
    description='File:',
    layout=widgets.Layout(width=dropdown_width)
)

def update_dropdowns(*args):
    # Filter files based on selected date
    date_filtered_files = [fn for fn in filenames if date_dropdown.value in fn]

    # Update Source dropdown
    available_sources = sorted(set(fn.split('_')[0] for fn in date_filtered_files))
    source_dropdown.options = available_sources
    if source_dropdown.value not in available_sources:
        source_dropdown.value = available_sources[0] if available_sources else None

    # Update Type dropdown
    available_types = sorted(set(fn.split('_')[1] for fn in date_filtered_files), reverse=True)
    type_dropdown.options = available_types
    if type_dropdown.value not in available_types:
        type_dropdown.value = available_types[0] if available_types else None

    # Update Period dropdown
    available_periods = sorted(set(fn.split('_')[2] for fn in date_filtered_files), reverse=True)
    period_dropdown.options = available_periods
    if period_dropdown.value not in available_periods:
        period_dropdown.value = available_periods[0] if available_periods else None

    # Update file options
    update_file_options()

def update_file_options(*args):
    filtered_files = [
        fn for fn in filenames
        if (date_dropdown.value in fn and
            source_dropdown.value == fn.split('_')[0] and
            type_dropdown.value == fn.split('_')[1] and
            period_dropdown.value == fn.split('_')[2])
    ]
    file_dropdown.options = filtered_files
    if filtered_files:
        file_dropdown.value = filtered_files[0]  # Set initial value to the first match
    else:
        file_dropdown.value = None

# Bind event handlers
date_dropdown.observe(update_dropdowns, 'value')
source_dropdown.observe(update_file_options, 'value')
type_dropdown.observe(update_file_options, 'value')
period_dropdown.observe(update_file_options, 'value')

# Display all dropdowns
display(date_dropdown, source_dropdown, type_dropdown, period_dropdown, file_dropdown)

# Initialize dropdowns
update_dropdowns()

Dropdown(description='Date:', layout=Layout(width='510px'), options=('20240916', '20240914', '20240913', '2024…

Dropdown(description='Source:', layout=Layout(width='510px'), options=('RUI', 'RUT', 'SOX', 'SPX', 'SPX+DJIA+N…

Dropdown(description='Type:', layout=Layout(width='510px'), options=('stocks',), value='stocks')

Dropdown(description='Period:', layout=Layout(width='510px'), options=('2y',), value='2y')

Dropdown(description='File:', layout=Layout(width='510px'), options=(), value=None)

### Step 2. Filter Stocks

In [78]:
# @title Filter 1. Sorting {"run":"auto"}
cond0 = "(df['RS (%)'] > 0)" # @param ["(df['RS (%)'] > 0)"]
cond1 = "& (df['RS Rank (%)'] > 95)" # @param ["& (df['RS Rank (%)'] > 95)", "& (df['RS Rank (%)'] > 90)", "& (df['RS Rank (%)'] > 85)", "& (df['RS Rank (%)'] > 80)", ""]
cond2 = "& ((df['EPS RS (%)'] > 0) | (df['Rev RS (%)'] > 0))" # @param ["& ((df['EPS RS (%)'] > 0) | (df['Rev RS (%)'] > 0))", ""]
cond3 = "& (df['EPS RS (%)'] > 0)" # @param ["& (df['EPS RS (%)'] > 0)", "& (df['EPS RS (%)'] > 50)", "& (df['EPS RS (%)'] > 100)", "& (df['EPS RS (%)'] > 200)", "& (df['EPS RS (%)'] > 500)", ""]
cond4 = "& (df['Rev RS (%)'] > 0)" # @param ["& (df['Rev RS (%)'] > 0)", "& (df['Rev RS (%)'] > 5)", "& (df['Rev RS (%)'] > 10)", "& (df['Rev RS (%)'] > 20)", "& (df['Rev RS (%)'] > 50)", "& (df['Rev RS (%)'] > 100)", ""]
cond5 = "& ((df['TTM EPS'] > 0) & (df['TTM RPS'] > 0))" # @param ["& (df['TTM EPS'] > 0)", "& (df['TTM RPS'] > 0)", "& ((df['TTM EPS'] > 0) & (df['TTM RPS'] > 0))", ""]
cond6 = "& (df['Price'] < 100)" # @param ["& (df['Price'] < 50)", "& (df['Price'] < 100)", "& (df['Price'] < 200)", ""]
sorted_column = "RS (%)" # @param ["RS (%)","1 Week Ago","1 Month Ago","3 Months Ago","6 Months Ago","Price","MA10","MA30","Volume / VMA10","EPS RS (%)", "Rev RS (%)"]
ascending = False # @param {"type":"boolean"}
num_items = 10 # @param [10, 20, 30] {"type":"raw"}

NUM_CONDS = 7
cond = eval('+'.join([f"cond{i}" for i in range(NUM_CONDS)]))

selected_file = file_dropdown.value
source, kind, period, ma, date = selected_file.split('_')
date = date.replace('.csv', '')
print(f"Source: {source}\nType: {kind}\nPeriod: {period}\nMA: {ma}\nDate: {date}")

df = github.download_csv(selected_file)
df = df[eval(cond)]
df = df.sort_values(by=sorted_column, ascending=ascending)
df_top_f1 = df.head(num_items)
display(df_top_f1)

print_column(df_top_f1, 'Name')
print_column(df_top_f1, 'Ticker')

Source: TWSE+TPEX
Type: stocks
Period: 2y
MA: SMA
Date: 20240916


,Ticker,Name,Sector,Industry,RS (%),1 Week Ago,1 Month Ago,3 Months Ago,6 Months Ago,9 Months Ago,RS Rank (%),Price,MA10,MA30,Volume / VMA10,EPS RS (%),TTM EPS,Rev RS (%),TTM RPS,TTM PE
16,1540,喬福,Industrials,Specialty Industrial Machinery,93.69,101.00,80.18,-7.66,-13.16,-1.59,99.14,47.60,38.60,26.19,1.44,28.98,2.09,15.57,7.516,22.58
20,6609,瀧澤科,Technology,Electronic Components,89.18,79.80,23.12,8.65,-17.51,-17.87,98.92,81.80,59.78,45.58,1.24,31.38,2.45,12.54,32.536,36.69
23,2365,昆盈,Technology,Computer Hardware,87.39,105.73,152.61,77.09,37.75,7.97,98.76,55.30,61.19,38.20,0.23,6.32,0.45,6.16,4.094,124.44
27,8937,合騏,Consumer Cyclical,Recreational Vehicles,85.40,104.32,52.01,-11.78,-12.43,5.87,98.54,53.60,46.33,29.95,0.89,46.78,1.12,46.75,1.649,52.59
28,3535,晶彩科,Technology,Computer Hardware,83.77,99.70,150.16,15.68,-10.04,-12.18,98.49,64.10,71.03,42.23,0.63,144.09,1.28,127.30,8.837,49.61
38,2543,皇昌,Industrials,Engineering & Construction,73.16,89.94,85.89,86.07,118.48,109.80,97.95,68.40,64.68,49.71,0.51,177.83,4.99,13.69,46.896,13.97
45,4561,健椿,Industrials,Metal Fabrication,63.98,74.34,26.27,39.47,3.20,50.75,97.57,75.00,61.60,49.31,0.96,6.07,4.28,7.93,22.745,18.62
52,3379,彬台,Industrials,Specialty Industrial Machinery,56.49,72.49,86.64,22.66,-8.82,9.40,97.19,36.00,41.32,26.09,0.80,340.48,0.48,38.76,18.121,75.10
54,6215,和椿,Industrials,Specialty Industrial Machinery,53.95,66.84,68.99,67.37,-10.16,-9.86,97.09,66.50,64.90,51.21,1.02,64.79,1.27,15.27,13.352,53.54
79,3284,太普高,Industrials,Specialty Industrial Machinery,44.53,79.01,76.04,60.12,23.88,5.57,95.74,44.35,51.14,37.11,0.80,74.45,1.65,22.12,10.217,26.61


喬福, 瀧澤科, 昆盈, 合騏, 晶彩科, 皇昌, 健椿, 彬台, 和椿, 太普高
1540, 6609, 2365, 8937, 3535, 2543, 4561, 3379, 6215, 3284


In [79]:
# @title Filter 2. Increasing RS > 0 {"run":"auto"}
cond0 = "(df['RS (%)'] > 0)" # @param ["(df['RS (%)'] > 0)"]
cond1 = "& (df['RS (%)'] > df['1 Week Ago'])" # @param ["& (df['RS (%)'] > df['1 Week Ago'])", ""]
cond2 = "& (df['1 Week Ago'] > df['1 Month Ago'])" # @param ["& (df['1 Week Ago'] > df['1 Month Ago'])", "& (df['RS (%)'] > df['1 Month Ago'])", ""]
cond3 = "& (df['1 Month Ago'] > df['3 Months Ago'])" # @param ["& (df['1 Month Ago'] > df['3 Months Ago'])", ""]
cond4 = "& (df['3 Months Ago'] > df['6 Months Ago'])" # @param ["& (df['3 Months Ago'] > df['6 Months Ago'])", ""]
cond5 = "" # @param ["& (df['6 Months Ago'] > df['9 Months Ago'])", ""]
cond6 = "& (df['RS Rank (%)'] > 80)" # @param ["& (df['RS Rank (%)'] > 95)", "& (df['RS Rank (%)'] > 90)", "& (df['RS Rank (%)'] > 85)", "& (df['RS Rank (%)'] > 80)", ""]
cond7 = "& ((df['EPS RS (%)'] > 0) | (df['Rev RS (%)'] > 0))" # @param ["& ((df['EPS RS (%)'] > 0) | (df['Rev RS (%)'] > 0))", ""]
cond8 = "& (df['EPS RS (%)'] > 0)" # @param ["& (df['EPS RS (%)'] > 0)", "& (df['EPS RS (%)'] > 50)", "& (df['EPS RS (%)'] > 100)", "& (df['EPS RS (%)'] > 200)", "& (df['EPS RS (%)'] > 500)", ""]
cond9 = "& (df['Rev RS (%)'] > 0)" # @param ["& (df['Rev RS (%)'] > 0)", "& (df['Rev RS (%)'] > 5)", "& (df['Rev RS (%)'] > 10)", "& (df['Rev RS (%)'] > 20)", "& (df['Rev RS (%)'] > 50)", "& (df['Rev RS (%)'] > 100)", ""]
cond10 = "& ((df['TTM EPS'] > 0) & (df['TTM RPS'] > 0))" # @param ["& (df['TTM EPS'] > 0)", "& (df['TTM RPS'] > 0)", "& ((df['TTM EPS'] > 0) & (df['TTM RPS'] > 0))", ""]
cond11 = "& (df['Price'] < 100)" # @param ["& (df['Price'] < 50)", "& (df['Price'] < 100)", "& (df['Price'] < 200)", ""]
num_items = 10 # @param [10, 20, 30] {"type":"raw"}

NUM_CONDS = 12
cond = eval('+'.join([f"cond{i}" for i in range(NUM_CONDS)]))

selected_file = file_dropdown.value
source, kind, period, ma, date = selected_file.split('_')
date = date.replace('.csv', '')
print(f"Source: {source}\nType: {kind}\nPeriod: {period}\nMA: {ma}\nDate: {date}")

df = github.download_csv(selected_file)
df = df[eval(cond)]
df_top_f2 = df.head(num_items)
display(df_top_f2)

print_column(df_top_f2, 'Name')
print_column(df_top_f2, 'Ticker')

Source: TWSE+TPEX
Type: stocks
Period: 2y
MA: SMA
Date: 20240916


,Ticker,Name,Sector,Industry,RS (%),1 Week Ago,1 Month Ago,3 Months Ago,6 Months Ago,9 Months Ago,RS Rank (%),Price,MA10,MA30,Volume / VMA10,EPS RS (%),TTM EPS,Rev RS (%),TTM RPS,TTM PE
20,6609,瀧澤科,Technology,Electronic Components,89.18,79.80,23.12,8.65,-17.51,-17.87,98.92,81.80,59.78,45.58,1.24,31.38,2.45,12.54,32.536,36.69
124,2477,美隆電,Technology,Consumer Electronics,31.45,5.67,-2.97,-12.04,-16.25,-4.26,93.31,32.05,27.00,23.17,3.47,89.43,1.12,6.71,14.985,31.47
137,2888,新光金,Financial Services,Insurance - Life,28.08,25.92,2.17,-10.01,-20.86,-7.80,92.61,13.40,11.76,9.78,0.99,2.38,1.18,22.96,16.362,11.31
150,6133,金橋,Industrials,Electrical Equipment & Parts,26.09,16.31,4.59,-17.09,-28.66,-23.76,91.91,22.60,21.24,16.89,2.28,112.03,0.41,19.43,9.229,56.59
157,6103,合邦,Technology,Semiconductors,24.91,2.26,-7.09,-12.72,-25.43,-14.67,91.53,52.70,42.15,38.09,2.00,78.47,0.58,12.70,3.296,89.31
176,6727,亞泰金屬,Industrials,Specialty Industrial Machinery,21.57,5.37,-1.94,-7.07,-22.26,-15.56,90.50,84.00,68.70,64.48,3.42,12.25,6.02,2.67,54.903,14.62
203,3484,崧騰,Industrials,Electrical Equipment & Parts,18.38,9.30,0.03,-5.12,-14.43,0.45,89.04,64.00,55.51,52.09,1.11,21.99,3.19,7.74,48.574,19.40
234,4935,茂林-KY,Technology,Electronic Components,15.64,12.95,-2.71,-5.53,-20.23,-12.88,87.37,71.80,61.36,59.68,3.91,1.75,3.77,4.51,48.971,18.67
361,1727,中華化,Basic Materials,Chemicals,7.04,0.30,-15.01,-21.19,-21.43,-7.94,80.52,35.50,32.50,30.46,0.57,190.34,0.17,5.61,14.983,216.47


瀧澤科, 美隆電, 新光金, 金橋, 合邦, 亞泰金屬, 崧騰, 茂林-KY, 中華化
6609, 2477, 2888, 6133, 6103, 6727, 3484, 4935, 1727


In [80]:
# @title Filter 3. RS Breakout {"run":"auto"}
base = 0 # @param [0, 30, 60, 100] {"type":"raw"}
cond0 = "(df['RS (%)'] > base)" # @param ["(df['RS (%)'] > base)"]
cond1 = "" # @param ["& (df['1 Week Ago'] < base)",""]
cond2 = "& (df['1 Month Ago'] < base)" # @param ["& (df['1 Month Ago'] < base)",""]
cond3 = "& (df['3 Months Ago'] < base)" # @param ["& (df['3 Months Ago'] < base)",""]
cond4 = "& (df['6 Months Ago'] < base)" # @param ["& (df['6 Months Ago'] < base)",""]
cond5 = "& (df['9 Months Ago'] < base)" # @param ["& (df['9 Months Ago'] < base)",""]
cond6 = "& (df['RS Rank (%)'] > 80)" # @param ["& (df['RS Rank (%)'] > 95)", "& (df['RS Rank (%)'] > 90)", "& (df['RS Rank (%)'] > 85)", "& (df['RS Rank (%)'] > 80)", ""]
cond7 = "& (df['Volume / VMA10'] > 2)" # @param ["& (df['Volume / VMA10'] > 1.5)", "& (df['Volume / VMA10'] > 2)", "& (df['Volume / VMA10'] > 3)", "& (df['Volume / VMA10'] > 4)", ""]
cond8 = "& (df['Price'] > df['MA30'])" # @param ["& (df['Price'] > df['MA10'])", "& (df['Price'] > df['MA30'])", ""]
cond9 = "" # @param ["& (df['MA10'] > df['MA30'])",""]
cond10 = "& ((df['EPS RS (%)'] > 0) | (df['Rev RS (%)'] > 0))" # @param ["& ((df['EPS RS (%)'] > 0) | (df['Rev RS (%)'] > 0))", ""]
cond11 = "" # @param ["& (df['EPS RS (%)'] > 0)", "& (df['EPS RS (%)'] > 50)", "& (df['EPS RS (%)'] > 100)", "& (df['EPS RS (%)'] > 200)", "& (df['EPS RS (%)'] > 500)", ""]
cond12 = "& ((df['TTM EPS'] > 0) & (df['TTM RPS'] > 0))" # @param ["& (df['TTM EPS'] > 0)", "& (df['TTM RPS'] > 0)", "& ((df['TTM EPS'] > 0) & (df['TTM RPS'] > 0))", ""]
cond13 = "" # @param ["& (df['Rev RS (%)'] > 0)", "& (df['Rev RS (%)'] > 5)", "& (df['Rev RS (%)'] > 10)", "& (df['Rev RS (%)'] > 20)", "& (df['Rev RS (%)'] > 50)", "& (df['Rev RS (%)'] > 100)", ""]
cond14 = "" # @param ["& (df['Price'] < 50)", "& (df['Price'] < 100)", "& (df['Price'] < 200)", "& (df['Price'] < 500)", ""]
num_items = 10 # @param [10, 20, 30] {"type":"raw"}

NUM_CONDS = 15
cond = eval('+'.join([f"cond{i}" for i in range(NUM_CONDS)]))

selected_file = file_dropdown.value
source, kind, period, ma, date = selected_file.split('_')
date = date.replace('.csv', '')
print(f"Source: {source}\nType: {kind}\nPeriod: {period}\nMA: {ma}\nDate: {date}")

df = github.download_csv(selected_file)
df = df[eval(cond)]
df_top_f3 = df.head(num_items)
display(df_top_f3)

print_column(df_top_f3, 'Name')
print_column(df_top_f3, 'Ticker')

Source: TWSE+TPEX
Type: stocks
Period: 2y
MA: SMA
Date: 20240916


,Ticker,Name,Sector,Industry,RS (%),1 Week Ago,1 Month Ago,3 Months Ago,6 Months Ago,9 Months Ago,RS Rank (%),Price,MA10,MA30,Volume / VMA10,EPS RS (%),TTM EPS,Rev RS (%),TTM RPS,TTM PE
58,4974,亞泰,Technology,Electronic Components,52.09,23.88,-4.73,-10.56,-13.85,-8.01,96.87,121.00,83.87,75.10,5.87,8.33,5.74,11.50,49.706,20.03
63,8249,菱光,Industrials,Electrical Equipment & Parts,49.87,6.26,-14.65,-18.74,-15.21,-7.90,96.60,50.20,31.95,30.33,7.97,-43.22,1.96,28.79,32.374,28.16
124,2477,美隆電,Technology,Consumer Electronics,31.45,5.67,-2.97,-12.04,-16.25,-4.26,93.31,32.05,27.00,23.17,3.47,89.43,1.12,6.71,14.985,31.47
176,6727,亞泰金屬,Industrials,Specialty Industrial Machinery,21.57,5.37,-1.94,-7.07,-22.26,-15.56,90.50,84.00,68.70,64.48,3.42,12.25,6.02,2.67,54.903,14.62
220,8032,光菱,Technology,Computer Hardware,17.20,-11.05,-21.24,-19.47,-15.93,-9.56,88.13,38.55,29.28,29.26,8.32,31.16,1.21,-1.54,49.253,35.04
232,3003,健和興,Industrials,Electrical Equipment & Parts,15.85,17.49,-6.15,-8.94,-20.81,-9.91,87.48,96.50,84.07,77.46,2.41,8.13,3.69,1.03,26.838,26.72
234,4935,茂林-KY,Technology,Electronic Components,15.64,12.95,-2.71,-5.53,-20.23,-12.88,87.37,71.80,61.36,59.68,3.91,1.75,3.77,4.51,48.971,18.67
248,3597,映興,Technology,Electronic Components,14.83,8.44,-8.50,-13.77,-12.44,-17.61,86.62,30.55,27.00,25.03,2.89,-7.97,0.24,0.68,14.650,122.29
255,5438,東友,Industrials,Business Equipment & Supplies,14.51,-12.00,-19.52,-17.97,-15.21,-13.02,86.24,23.00,17.85,18.25,8.67,-284.10,0.16,1.22,14.514,158.12
364,4735,豪展,Healthcare,Medical Devices,6.91,12.92,-2.11,-22.12,-14.28,-8.87,80.36,45.65,43.24,38.52,2.20,153.08,0.40,14.59,22.359,111.50


亞泰, 菱光, 美隆電, 亞泰金屬, 光菱, 健和興, 茂林-KY, 映興, 東友, 豪展
4974, 8249, 2477, 6727, 8032, 3003, 4935, 3597, 5438, 4735


### Step 3. Visualize Filtered Stocks

In [116]:
# @title Plot 1. Mansfield Relative Strength Comparison {"run":"auto"}
source = "Filter 3. RS Breakout" # @param ["Filter 1. Sorting", "Filter 2. Increasing RS","Filter 3. RS Breakout","Major Global Stock Indices", "Sector Indices"]
backend = "mplfinance" # @param ["mplfinance","Plotly"]

import matplotlib.pyplot as plt
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display, clear_output

MAX_STOCK_SELECTION = 10
N_COLS = 8  # the number of columns of the grid layout for checkboxes

symbols = {
    'Filter 1. Sorting': sorting_tickers,
    'Filter 2. Increasing RS': increasing_rs_tickers,
    'Filter 3. RS Breakout': rs_breakout_tickers,
    'Major Global Stock Indices': major_indices,
    'Sector Indices': sector_indices,
}[source]()

checkboxes = cbs_create(symbols, MAX_STOCK_SELECTION)
checkbox_grid = cbs_with_grid(checkboxes, N_COLS)
btn_unselect_all = widgets.Button(description="Unselect All")
btn_unselect_all.on_click(lambda b: cbs_unselect_all(checkboxes))
btn_select_top = widgets.Button(description="Select Top 10")
btn_select_top.on_click(lambda b: cbs_select_top(checkboxes, MAX_STOCK_SELECTION))

period_dropdown = create_period_dropdown()
interval_dropown = create_interval_dropdown()

cmp_theme_dropdown = {
    'mplfinance': create_style_dropdown('Comparison Theme:', 'charles'),
    'Plotly': create_template_dropdown('Comparison Theme:', 'plotly_dark'),
}[backend]
btn_plot = widgets.Button(description="Generate Plot")

out_msg, out_fig = widgets.Output(), widgets.Output()

ui = widgets.VBox([
    checkbox_grid,
    widgets.HBox([btn_unselect_all, btn_select_top]),
    widgets.VBox([period_dropdown, interval_dropown]),
    widgets.VBox([cmp_theme_dropdown, btn_plot]),
    out_msg, out_fig
])
display(ui)

import vistock.mpl.mansfield as mpl_mansfield
import vistock.plotly.mansfield as ply_mansfield

rs_cmp = {
    'mplfinance': mpl_mansfield.RelativeStrengthLines,
    'Plotly': ply_mansfield.RelativeStrengthLines,
}[backend]

def on_checkbox_change(change):
    selected_count = sum([cb.value for cb in checkboxes])
    if selected_count > MAX_STOCK_SELECTION:
        # Uncheck the last checked box if selection exceeds limit
        changed_checkbox = change['owner']
        changed_checkbox.value = False
        with out_msg:
            out_fig.clear_output()
            print(f"Only {MAX_STOCK_SELECTION} stocks can be selected at most.")

# Bind the checkbox change event to the function
for checkbox in checkboxes:
    checkbox.observe(on_checkbox_change, names='value')

def on_plot_click(b):
    symbols = cbs_get_selected(checkboxes)
    if not symbols:
        with out_msg:
            out_fig.clear_output()
            print("No stocks selected. Please select at least one stock.")
        return
    with out_fig:
        out_msg.clear_output()
        clear_output()
        interval = interval_dropown.value
        period = period_dropdown.value
        if rs_cmp is mpl_mansfield.RelativeStrengthLines:
            rs_cmp.plot(symbols, interval=interval, period=period,
                        style=cmp_theme_dropdown.value,
                        color_cycle=plt.cm.Paired.colors)
        else: # Plotly
            rs_cmp.plot(symbols, interval=interval, period=period,
                        template=cmp_theme_dropdown.value,
                        colorway=px.colors.qualitative.Set3)

btn_plot.on_click(on_plot_click)

if backend == 'Plotly':
    enable_plotly_in_cell()


In [ ]:
# @title Plot 2. Mansfield Stock Chart {"run":"auto"}
source = "Taiwan Stocks" # @param ["Filter 1. Sorting", "Filter 2. Increasing RS","Filter 3. RS Breakout","Major Global Stock Indices", "U.S. Listed Stocks", "Taiwan Stocks"]
backend = "mplfinance" # @param ["mplfinance","Plotly"]

import functools as ft
import ipywidgets as widgets
from IPython.display import display, clear_output
import yfinance as yf

import vistock.stock_indices as si
import vistock.tw as tw

symbols = {
    'Filter 1. Sorting': sorting_tickers,
    'Filter 2. Increasing RS': increasing_rs_tickers,
    'Filter 3. RS Breakout': rs_breakout_tickers,
    'Major Global Stock Indices': major_indices,
    'U.S. Listed Stocks': ft.partial(si.get_tickers, 'USLS'),
    'Taiwan Stocks': ft.partial(si.get_tickers, 'TWSE+TPEX+ESB'),
}[source]()
if source == 'Taiwan Stocks':
    symbols = [f"{tw.stock_name(s)} {s}" for s in symbols]

dropdowns, layout = create_search_dropdowns(symbols, 1)

period_dropdown2 = create_period_dropdown('2y')
interval_dropown2 = create_interval_dropdown('1wk')
stock_theme_dropdown = {
    'mplfinance': create_style_dropdown('Stock Theme:', 'yahoo'),
    'Plotly': create_template_dropdown('Stock Theme:', 'plotly'),
}[backend]

btn_plot_prc = widgets.Button(description="Price/RS/Volume Chart",
                              layout=widgets.Layout(width='168px'))
btn_plot_fin = widgets.Button(description="Financial Chart",
                              layout=widgets.Layout(width='168px'))
btn_report_q = widgets.Button(description="Quarterly Report",
                              layout=widgets.Layout(width='168px'))
btn_report_a = widgets.Button(description="Annual Report",
                              layout=widgets.Layout(width='168px'))
btn_clear_last = widgets.Button(description="Clear Last")
btn_clear_all = widgets.Button(description="Clear All")
#outputs = widgets.VBox()

ui = widgets.VBox([
    layout,
    period_dropdown2, interval_dropown2, stock_theme_dropdown,
    widgets.HBox([btn_plot_prc, btn_plot_fin]),
    widgets.HBox([btn_report_q, btn_report_a]),
    widgets.HBox([btn_clear_last, btn_clear_all]),
    outputs
])
display(ui)

import vistock.mpl.mansfield as mpl_mansfield
import vistock.plotly.mansfield as ply_mansfield

stock_chart = {
    'mplfinance': mpl_mansfield.StockChart,
    'Plotly': ply_mansfield.StockChart,
}[backend]

def get_symbols():
    symbols = get_dropdowns_selected_options(dropdowns)
    if source == 'Taiwan Stocks':
        symbols = [s.split()[0] for s in symbols]
    return symbols

def on_plot_prc_click(b):
    selected = get_symbols()
    new_output = widgets.Output()
    with new_output:
        interval = interval_dropown2.value
        period = period_dropdown2.value
        if not selected:
            print("No Stock Selected!")
        elif stock_chart is mpl_mansfield.StockChart:
            stock_chart.plot(selected[0], interval=interval, period=period,
                             style=stock_theme_dropdown.value,
                             legend_loc='upper left')
        else: # Plotly
            stock_chart.plot(selected[0], interval=interval, period=period,
                             template=stock_theme_dropdown.value)
    outputs.children = (new_output,) + outputs.children

import vistock.mpl.financials as mpl_fin
import vistock.plotly.financials as ply_fin

fin_chart = {
    'mplfinance': mpl_fin,
    'Plotly': ply_fin,
}[backend]

def on_plot_fin_click(b):
    selected = get_symbols()
    new_output = widgets.Output()
    with new_output:
        if not selected:
            print("No Stock Selected!")
        elif fin_chart is mpl_fin:
            fin_chart.plot(selected[0], style=stock_theme_dropdown.value)
        else: # Plotly
            fin_chart.plot(selected[0], template=stock_theme_dropdown.value)
    outputs.children = (new_output,) + outputs.children

def on_report_q_click(b):
    selected = get_symbols()
    new_output = widgets.Output()
    with new_output:
        if not selected:
            print("No Stock Selected!")
        else:
            symbol = tw.as_yfinance(selected[0])
            ticker = tw.as_yfinance(symbol)
            print(f"\n{symbol} Quarterly Financials:")
            display(yf.Ticker(ticker).quarterly_financials)
    outputs.children = (new_output,) + outputs.children

def on_report_a_click(b):
    selected = get_symbols()
    new_output = widgets.Output()
    with new_output:
        if not selected:
            print("No Stock Selected!")
        else:
            symbol = tw.as_yfinance(selected[0])
            ticker = tw.as_yfinance(symbol)
            print(f"\n{symbol} Annual Financials:")
            display(yf.Ticker(ticker).financials)
    outputs.children = (new_output,) + outputs.children

def on_clear_last_click(b):
    if outputs.children:
        children = list(outputs.children)
        children.pop(0)
        outputs.children = tuple(children)

def on_clear_all_click(b):
    outputs.children = ()

btn_plot_prc.on_click(on_plot_prc_click)
btn_plot_fin.on_click(on_plot_fin_click)
btn_report_q.on_click(on_report_q_click)
btn_report_a.on_click(on_report_a_click)
btn_clear_last.on_click(on_clear_last_click)
btn_clear_all.on_click(on_clear_all_click)

if backend == 'Plotly':
    enable_plotly_in_cell()


Dropdown(description='Period:', index=1, options=('1y', '2y', '5y'), value='2y')

Dropdown(description='Interval:', index=1, options=('1d', '1wk'), value='1wk')

Dropdown(description='Stock Theme:', index=2, options=('default', 'classic', 'yahoo', 'charles', 'tradingview'…

VBox()